# Chloe Wang, Qihan Guan, Michael Harris # 
### BAX-422 Group 5 ###
### March 9, 2021 ### 

In [3]:
pip install pyzipcode 

  Created wheel for pyzipcode: filename=pyzipcode-3.0.1-py3-none-any.whl size=1932419 sha256=37d9db666e6db5b2b6a71ac09875e78788eff76a6ee3eb7e22c1bc4658a52904
  Stored in directory: c:\users\dvcgu\appdata\local\pip\cache\wheels\f6\01\8e\7424c7e867eb5db7bb9531b09b4d968f23e5ac45ec5e7c3cb3
Successfully built pyzipcode
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import packages
from bs4 import BeautifulSoup, Tag
from urllib.request import Request, urlopen
from urllib.request import urlopen
import requests
import time
import warnings
import requests
import os
import json
import re
import codecs
import urllib.request, urllib.error, urllib.parse
import shutil
from lxml import html
from time import sleep
from itertools import zip_longest
from pyzipcode import ZipCodeDatabase
import pandas as pd
import numpy as np

In [3]:
# Ask Input for Car make.
make = str(input("Enter Make(Press enter directly to omit): ")).lower()

# Ask Input for Model
model = str(input("Enter Model(Press enter directly to omit): ")).lower()

# Ask Input for Minimum Year
min_year = ''
while True: 
  year1 = int(input("Please enter vehicle minimum year: (1981-2021): ")) 
  min_year = str(year1)
  if year1 not in range(1981,2022):
    print("**Please enter a valid year**")
    continue
  else:
    break


# Ask Input for Maximum Year
max_year = ''
while True: 
  year2 = int(input("Please enter vehicle maximum year: (1981-2021): ")) 
  max_year = str(year2)
  if year2 not in range(1981,2022):
    print("**Please enter a valid year**")
    continue
  elif year2 < int(min_year):
    print("**Maximum year cannot be older than minimum year**")
    continue
  else:
    break

# Ask Input for Zip Code
url_geo=''
while True: 
  zipcode = str(input("Please enter your 5-digit Zip Code: ")) 
  zipdb = ZipCodeDatabase()
  try: 
    geoinfo = zipdb[zipcode]
    url_geo = str(geoinfo.city).lower().replace(" ","-") +"-"+str(geoinfo.state).lower()+"-"+str(zipcode)
  except: 
    print("**Please enter a valid zipcode**") 
    continue
  else:
    break
print(url_geo)
# Ask Input for Radius
radius=''
while True: 
  radius = str(input("Please enter your search radius in miles (10/25/50/75/100/200/300/400/500/ 0 for any): ")) 
  if radius not in ['10','25','50','75','100','200','300','400','500','0']:
    print("**Please enter a valid radius**")
    continue
  else:
    break


Enter Make(Press enter directly to omit): BMW
Enter Model(Press enter directly to omit): 530i
Please enter vehicle minimum year: (1981-2021): 2017
Please enter vehicle maximum year: (1981-2021): 2021
Please enter your 5-digit Zip Code: 90001
los-angeles-ca-90001
Please enter your search radius in miles (10/25/50/75/100/200/300/400/500/ 0 for any): 50


In [4]:
# Compose the URL based on user input. Set to 100 items per page to miminze get requests
url = str("https://www.autotrader.com/cars-for-sale/all-cars/"+make+"/"+model+"/"+url_geo+"?dma=&searchRadius="+
radius+"&location=&marketExtension=include&startYear="+min_year+"&endYear="+max_year+"&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100")

print(url)

https://www.autotrader.com/cars-for-sale/all-cars/bmw/530i/los-angeles-ca-90001?dma=&searchRadius=50&location=&marketExtension=include&startYear=2017&endYear=2021&isNewSearch=true&showAccelerateBanner=false&sortBy=relevance&numRecords=100


### Save all the listed vehicle URLs under the user input criteria

In [5]:
# Scrape All URLs from Search:
header = ({'User-Agent':'Mozilla/5.0'}) 

# Initialize a list to save the item urls
url_list = []

# Loop thorugh all the pages available 
i = 0 # set a intial counter 0
while True:
    URL = url + '&firstRecord=' + str(i*100) #Autotrader uses a multiplier of num_records_per_page to navigate to different pages
    response = requests.get(URL, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')
    pagebuttons = soup.find_all('li', {'class':'active'})
    out_of_bound = True
    
    #Check in page number out-of-bound
    for buttons in pagebuttons:
        if(len(buttons.find_all('a', {'role':'button'}))!=0):
            out_of_bound = False
    
    if out_of_bound == True: # Stop if page number out-of-bound
        break
    #else, scrape url of each listed vehicle for sale
    else:
        items = soup.findAll('div',{"class":"display-flex justify-content-between"})
        for item in items:
            a = item.find("a")
            if not a:
                continue
            url_simplify = 'https://www.autotrader.com' + a['href']
            url_list.append(url_simplify + "\n")
            print(url_simplify)
        #set a time lapse
        time.sleep(4)
        
    #increment counter
    i = i+1
    
  

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=577218088&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D0%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2017&numRecords=100&firstRecord=0&endYear=2021&modelCodeList=530I&makeCodeList=BMW&searchRadius=50&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=578985771&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D0%26ma

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=577218088&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D100%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2017&numRecords=100&firstRecord=100&endYear=2021&modelCodeList=530I&makeCodeList=BMW&searchRadius=50&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=578020152&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D10

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=577218088&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D200%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2017&numRecords=100&firstRecord=200&endYear=2021&modelCodeList=530I&makeCodeList=BMW&searchRadius=50&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=571817530&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D20

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=577218088&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D300%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2017&numRecords=100&firstRecord=300&endYear=2021&modelCodeList=530I&makeCodeList=BMW&searchRadius=50&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=576006028&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D30

https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=577218088&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%26state%3DCA%26firstRecord%3D400%26marketExtension%3Dinclude%26relevanceConfig%3Ddefault%26searchRadius%3D50%26isNewSearch%3Dtrue&startYear=2017&numRecords=100&firstRecord=400&endYear=2021&modelCodeList=530I&makeCodeList=BMW&searchRadius=50&makeCode1=BMW&modelCode1=530I&clickType=alpha
https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=574971455&zip=90001&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D90001%26city%3DLos%2BAngeles%26startYear%3D2017%26numRecords%3D100%26incremental%3Dall%26endYear%3D2021%26modelCodeList%3D530I%26makeCodeList%3DBMW%26sortBy%3Drelevance%26location%3D%255Bobject%2BObject%255D%2

In [6]:
#Check the total number of URLs saved
len(url_list)

510

### Open all the URLs and save the listing vehicles' pages in a directory

In [7]:
#Compose a filename based on the user input
file_name = make+'_'+model+'_'+min_year+'_'+max_year+'_'+str(zipcode)+'_'+str(radius)+'miles'

#Write all listed vehicle's url to file_name.txt
with open(os.getcwd()+'/'+file_name+'.txt', 'w+') as filehandle:
    for item_url in url_list:
        filehandle.write('%s' % item_url)

In [8]:
#read the txt file and save the listed vehicles' pages as html files
cars = open(os.getcwd()+'/'+file_name+'.txt', 'r').read().split('\n')
headers = {'User-Agent':'Mozilla/5.0'}  

#create path for 'file_name' folder
path = os.path.join(os.getcwd() , file_name)
if not os.path.exists(path):
    os.makedirs(path)

try:
    #Download each of the pages(URLs) into the folder 'file_name'
    for URL in cars[0:243]:
        listing_id = re.search(r'\?listingId=([0-9]+)&', URL).group(1) #Use listing ID as name for each html file
        page = requests.get(URL, headers=headers) 
        with open(os.path.join(path, str(listing_id)+'.html'), 'w+', encoding="utf-8") as filehandle:
            filehandle.write(page.text)

        #put a 3 sec pause between queries
        time.sleep(2)
except:
    print('Problem with connection...')
        

In [9]:
#Check how many files saved successfully
list = os.listdir(path) 
print(len(list))

232


### Open each saved html file and save the data we want in a pd dataframe
For each listed vehicle, we will save the following information:

Listing ID, Vehicle Condition, Year, Make, Model, Trim Level, List Price, Mileage, Transimission, Drivetrain(FWD/RWD/AWD), Exterior Color, Interior Color, Engine Type, City MPG, Highway Mpg, Seller Type(Private or Dealer), Seller Name, Seller Rating, Seller Number of Reviews, Seller Street Address, Seller City, Seller State, Seller Zipcode, Seller Phone Number

In [10]:
#initialize a data frame to save 

vehicle_info = pd.DataFrame(columns=['Listing ID','Condition', 'Year', 'Make', 'Model','Trim Level','List Price','Mileage',
                                    'Exterior Color', 'Interior Color','Engine Type','Transmission','Drivetrain(FWD/RWD/AWD)', 
                                     'City MPG', 'Highway MPG', 'Seller Type', 'Seller Name','Seller Rating',
                                    'Number of Reviews','Seller Street Address', 'Seller City','Seller State',
                                     'Seller Zipcode','Seller Phone Number'])

for i in range(0, len(list)):
    try:
        page = list[i]
        listing_id = re.search(r'([0-9]+)\.html',list[i]).group(1) #Get Listing ID
        page_file = open(path + '\\' + page, 'r', encoding="utf-8").read() 
        soup = BeautifulSoup(page_file, 'html.parser')

        #selector command for Vehicle Condition
        try:
            title_raw = soup.find('h1', class_ ='text-bold text-size-400 text-size-sm-700').text
            condition = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(1)
        except:
            condition = None
            
        #selector command for Vehicle Year
        try:
            year = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(2)
        except:
            year = None

        #selector command for Vehicle Make
        try:
            make = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(3)
        except:
            make = None
        
        #selector command for Vehicle Model
        try:
            model = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+)',title_raw).group(4)
        except:
            model = None
        
        #selector command for Vehicle Trim Level
        try:
            trim = re.search(r'([UCN][a-z]+) ([0-9]{4}) ([A-Za-z]+) ([A-Za-z0-9]+) (.+)',title_raw).group(5)
        except:
            trim = None
        
        #selector command for List Price
        #In most cases, a net price will be displayed
        try:
            price = int(soup.find('span',{'class':'first-price'}).text.replace(',',''))
        except:#For new vehicles, some of them will display only the MSRP as price
            try:
                price_raw = soup.find('span', {'class':'padding-left-2 padding-bottom-1 text-normal second-price'}).find('span', {'class':'text-bold'}).text
                price = int(price_raw.replace(',',''))  
            except:
                price = None
        
        #selector command for Vehicle Mileage
        try:
            #find the element following the mileage icon
            m = soup.find('div', {'style':'background:url(https://www.autotrader.com/content/dam/autotrader/findcar/vdp/specifications/all_spec_icons.svg) -191px -74px;height:25px;width:30px'})
            mileage_raw = re.search(r'(.+) miles',m.parent.find_next_sibling('div').text).group(1)
            mileage = int(mileage_raw.replace(',',''))
        except:
            mileage = None
        
        #selector command for Exterior Color
        try:
            #find the element following the exterior color icon
            excol = soup.find_all('div', {'class':'swatch'})[0]
            extcolor = re.search(r'(.+) Exterior',excol.parent.parent.nextSibling.text).group(1)
        except:
            extcolor = None
        
        #selector command for Interior Color
        try:
            #find the element following the interior color icon
            incol = soup.find_all('div', {'class':'swatch'})[1]
            intcolor = re.search(r'(.+) Interior',incol.parent.parent.nextSibling.text).group(1)
        except:
            intcolor = None
        
        #selector command for Engine Type
        try:
            eng = soup.find('div', {'aria-label':"ENGINE_DESCRIPTION"})
            engine = eng.parent.find_next_sibling('div').text
        except:
            engine = None
        
        #selector command for Transmission
        try:
            trans = soup.find('div', {'aria-label':"TRANSMISSION"})
            transmission = trans.parent.find_next_sibling('div').text
        except:
            transmission = None
        
        #selector command for Drivetrain
        try:
            drive = soup.find('div', {'aria-label':"DRIVE TYPE"})
            drivetrain = drive.parent.find_next_sibling('div').text
        except:
            drivetrain = None
            
        #selector command for City MPG
        try:
            mpg = soup.find('div', {'aria-label':"MPG"}).parent.find_next_sibling('div').text
            city_mpg = re.search(r'([0-9]+) City / ([0-9]+) Highway',mpg).group(1)
        except:
            city_mpg = None
            
        #selector command for Highway MPG
        try:
            mpg = soup.find('div', {'aria-label':"MPG"}).parent.find_next_sibling('div').text
            highway_mpg = re.search(r'([0-9]+) City / ([0-9]+) Highway',mpg).group(2)
        except:
            highway_mpg = None
            
        #Get Seller's type (either private seller or dealer)
        #if find the h2 class for 'private seller' heading, then it's private seller
        if(len(soup.find_all('h2', {'class':'text-bold padding-bottom-3'}))!=0):
            seller_type = 'Private Seller'
        #otherwise, the seller is a dealer    
        else:
            seller_type = 'Dealer'
            
        #Get Seller Name
        if seller_type == 'Dealer':
            try:
                seller_name = soup.find('h2', {'class':"text-left text-bold text-size-20 display-inline"}).text
            except:
                try:
                    seller_name = soup.find('h2', {'class':"text-left text-bold text-size-20"}).text
                except:
                    seller_name = None
        else:
            seller_name = None #Set to 'None' for private sellers because Not Applicable
            
        #Get Seller Rating
        if seller_type == 'Dealer':
            try:
                seller_rating = float(soup.find('span', {'class': "text-bold text-subdued"}).text)
            except:
                seller_rating = None
        else:
            seller_rating = None #Set to 'None' for private sellers because Not Applicable
        
        #Get Number of Reviews for the Seller
        if seller_type == 'Dealer':
            try:
                num_reviews_raw = soup.find('a', {'class': "cursor-pointer padding-left-1"}).text
                num_reviews = int(re.search(r'([0-9]+)', num_reviews_raw).group(1))
            except:
                num_reviews = 0 
        else:
            num_reviews = None #Set to 'None' for private sellers because Not Applicable
        
        #Get Seller Street Address
        addrs = soup.find_all('div', {'data-cmp': "address"})
        address = ''
        for addr in addrs:
            for line in addr:
                address = address + line.text
        try:
            seller_street = re.search(r'([0-9]+ [A-Z].+), [A-Za-z\s]+, ', address).group(1)
        except:
            seller_street = None
            
        #Get Seller City
        try:
            seller_city = re.search(r'(.*,)?\s?(.*), [A-Z]{2} [0-9]{5}', address).group(2)
        except:
            seller_city = None  
            
        #Get Seller State
        try:
            seller_state = re.search(r', ([A-Z]{2}) ([0-9]{5})', address).group(1)
        except:
            seller_state = None
        
        #Get Seller Zipcode
        try:
            seller_zip = re.search(r', ([A-Z]{2}) ([0-9]{5})', address).group(2)
        except:
            seller_zip = None
        
        #Get Seller Phone Number
        try:
            seller_phone = soup.find('span', {'data-cmp':'phoneNumber'}).text
        except:
            seller_phone = None
            
        #save this iteration's result to the i-th row of the dataframe
        vehicle_info.loc[i] = [listing_id, condition, year, make, model, trim, price, mileage, extcolor, intcolor, engine, transmission, drivetrain, city_mpg, 
                       highway_mpg, seller_type, seller_name, seller_rating, num_reviews, seller_street, seller_city, seller_state,
                       seller_zip, seller_phone]

           
    except:
        print('Error with: ' + page)

### View Saved Data

In [11]:
vehicle_info

,Listing ID,Condition,Year,Make,Model,Trim Level,List Price,Mileage,Exterior Color,Interior Color,...,Highway MPG,Seller Type,Seller Name,Seller Rating,Number of Reviews,Seller Street Address,Seller City,Seller State,Seller Zipcode,Seller Phone Number
0,538531883,Used,2020,BMW,530i,None,59615,38,0416 Carbon Black Metallic,None,...,33,Dealer,Century West BMW,4.8,1790,4245 Lankershim Boulevard,Universal City,CA,91608,(747) 203-7099
1,539875817,Used,2020,BMW,530i,None,47688,7744,Silver,Black,...,33,Dealer,Beverly Hills BMW,4.3,2358,5070 Wilshire Blvd,Los Angeles,CA,90036,(323) 238-2328
2,548362207,Used,2018,BMW,530i,None,30750,34778,Black,Beige,...,34,Dealer,Puente Hills Mitsubishi,4.1,360,17665 CASTLETON ST,CITY OF INDUSTRY,CA,91748,(833) 375-4875
3,554804112,Used,2020,BMW,530i,None,58305,8,0c10 Mediterranean Blue Metallic,None,...,33,Dealer,Century West BMW,4.8,1790,4245 Lankershim Boulevard,Universal City,CA,91608,(747) 203-7099
4,555046601,Used,2017,BMW,530i,None,25900,29726,Jet Black,Black,...,34,Dealer,California Beemers,4.9,310,2845 HARBOR BLVD,COSTA MESA,CA,92626,(949) 942-6096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,579448998,Used,2021,BMW,530i,None,49991,3543,Jet Black,Cognac Perforated Sensatec,...,33,Dealer,Bob Smith BMW Mini,5,1,24500 Calabasas Rd,Calabasas,CA,91302,(747) 263-0918
228,579452141,Certified,2018,BMW,530i,None,31589,19514,Black,Black Sensa,...,34,Dealer,South Bay BMW,4.6,1608,18800 HAWTHORNE BLVD,TORRANCE,CA,90504,(424) 323-3393
229,579482240,Used,2018,BMW,530i,None,30980,23895,Mineral White Metallic,Black,...,34,Dealer,Valencia BMW,4.7,2335,23435 Valenica Blvd,Valencia,CA,91355,(661) 383-0283
230,579527433,Used,2018,BMW,530i,None,28725,40313,Alpine White,Black,...,34,Dealer,BMW of Buena Park,4.4,2588,6750 AUTO CENTER DR,BUENA PARK,CA,90621,(657) 325-0354


### Save Data to MongoDB

In [41]:
#Connect to MongoDB
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

db = client[file_name] #use the user input criteria to compose a database name
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bmw_530i_2017_2021_90001_50miles')

Save data from dataframe to MongoDB collection row by row:

In [42]:
#Show database name
print(file_name)
mycollection = db[file_name]
print(mycollection)

bmw_530i_2017_2021_90001_50miles
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bmw_530i_2017_2021_90001_50miles'), 'bmw_530i_2017_2021_90001_50miles')


In [43]:
for i in vehicle_info.index:
    #If seller is a dealer, insert all info
    if vehicle_info['Seller Type'][i] == 'Dealer':
        listing_id = int(vehicle_info['Listing ID'][i])
        try:
            Condition = str(vehicle_info['Condition'][i])
        except:
            Condition = None
        try:
            Year = int(vehicle_info['Year'][i])
        except:
            Year = None
        try:
            Make = str(vehicle_info['Make'][i])
        except:
            Make = None
        try:
            Model = str(vehicle_info['Model'][i])
        except:
            Model = None
        try:
            TrimLevel = str(vehicle_info['Trim Level'][i])
        except:
            TrimLevel = None
        try:
            ListPrice = int(vehicle_info['List Price'][i])
        except:
            ListPrice = None
        try:
            Mileage = int(vehicle_info['Mileage'][i])
        except:
            Mileage = None
        try:
            ExteriorColor = str(vehicle_info['Exterior Color'][i])
        except:
            ExteriorColor = None
        try:
            InteriorColor = str(vehicle_info['Interior Color'][i])
        except:
            InteriorColor = None
        try:
            EngineType = str(vehicle_info['Engine Type'][i])
        except:
            EngineType = None
        try:
            Transmission = str(vehicle_info['Transmission'][i])
        except:
            Transmission = None
        try:
            Drivetrain = str(vehicle_info['Drivetrain(FWD/RWD/AWD)'][i])
        except:
            Drivetrain = None
        try:
            CityMPG = int(vehicle_info['City MPG'][i])
        except:
            CityMPG = None
        try:
            HighwayMPG = int(vehicle_info['Highway MPG'][i])
        except:
            HighwayMPG = None
        SellerType = str(vehicle_info['Seller Type'][i])
        try:
            SellerName = str(vehicle_info['Seller Name'][i])
        except:
            SellerName = None
        try:
            SellerRating = float(vehicle_info['Seller Rating'][i])
        except:
            SellerRating = None
        try:
            NumberofReviews = int(vehicle_info['Number of Reviews'][i])
        except:
            NumberofReviews = None
        try:
            SellerStreetAddress = str(vehicle_info['Seller Street Address'][i])
        except:
            SellerStreetAddress = None
        try:
            SellerCity = str(vehicle_info['Seller City'][i])
        except:
            SellerCity = None
        try:
            SellerState = str(vehicle_info['Seller State'][i])
        except:
            SellerState = None
        try:
            SellerZipcode = str(vehicle_info['Seller Zipcode'][i])
        except:
            SellerZipcode = None
        try:
            SellerPhoneNumber = str(vehicle_info['Seller Phone Number'][i])
        except:
            SellerPhoneNumber = None
        
        mydict = {
            '_id' : listing_id,
            'Condition': Condition,
            'Year': Year,
            'Make': Make,
            'Model': Model,
            'Trim Level': TrimLevel,
            'List Price': ListPrice,
            'Mileage': Mileage,
            'Exterior Color': ExteriorColor,
            'Interior Color': InteriorColor,
            'Engine Type': EngineType,
            'Transmission': Transmission,
            'Drivetrain': Drivetrain,
            'City MPG': CityMPG,
            'Highway MPG': HighwayMPG,
            'Seller Type': SellerType,
            'Seller Name': SellerName,
            'Seller Rating': SellerRating,
            'Number of Reviews': NumberofReviews,
            'Seller Street Address': SellerStreetAddress,
            'Seller City': SellerCity,
            'Seller State': SellerState,
            'Seller Zipcode': SellerZipcode,
            'Seller Phone Number': SellerPhoneNumber 
        }
        
        #insert this listed vehicle's data to MongoDB database 
        mycollection.insert_one(mydict)
    
    #If seller is a private seller, do not save Seller Name, Seller Rating or Number of Reviews
    else:
        listing_id = int(vehicle_info['Listing ID'][i])
        try:
            Condition = str(vehicle_info['Condition'][i])
        except:
            Condition = None
        try:
            Year = int(vehicle_info['Year'][i])
        except:
            Year = None
        try:
            Make = str(vehicle_info['Make'][i])
        except:
            Make = None
        try:
            Model = str(vehicle_info['Model'][i])
        except:
            Model = None
        try:
            TrimLevel = str(vehicle_info['Trim Level'][i])
        except:
            TrimLevel = None
        try:
            ListPrice = int(vehicle_info['List Price'][i])
        except:
            ListPrice = None
        try:
            Mileage = int(vehicle_info['Mileage'][i])
        except:
            Mileage = None
        try:
            ExteriorColor = str(vehicle_info['Exterior Color'][i])
        except:
            ExteriorColor = None
        try:
            InteriorColor = str(vehicle_info['Interior Color'][i])
        except:
            InteriorColor = None
        try:
            EngineType = str(vehicle_info['Engine Type'][i])
        except:
            EngineType = None
        try:
            Transmission = str(vehicle_info['Transmission'][i])
        except:
            Transmission = None
        try:
            Drivetrain = str(vehicle_info['Drivetrain(FWD/RWD/AWD)'][i])
        except:
            Drivetrain = None
        try:
            CityMPG = int(vehicle_info['City MPG'][i])
        except:
            CityMPG = None
        try:
            HighwayMPG = int(vehicle_info['Highway MPG'][i])
        except:
            HighwayMPG = None
        SellerType = str(vehicle_info['Seller Type'][i])
        try:
            SellerStreetAddress = str(vehicle_info['Seller Street Address'][i])
        except:
            SellerStreetAddress = None
        try:
            SellerCity = str(vehicle_info['Seller City'][i])
        except:
            SellerCity = None
        try:
            SellerState = str(vehicle_info['Seller State'][i])
        except:
            SellerState = None
        try:
            SellerZipcode = str(vehicle_info['Seller Zipcode'][i])
        except:
            SellerZipcode = None
        try:
            SellerPhoneNumber = str(vehicle_info['Seller Phone Number'][i])
        except:
            SellerPhoneNumber = None
        
        mydict = {
            '_id' : listing_id,
            'Condition': Condition,
            'Year': Year,
            'Make': Make,
            'Model': Model,
            'Trim Level': TrimLevel,
            'List Price': ListPrice,
            'Mileage': Mileage,
            'Exterior Color': ExteriorColor,
            'Interior Color': InteriorColor,
            'Engine Type': EngineType,
            'Transmission': Transmission,
            'Drivetrain': Drivetrain,
            'City MPG': CityMPG,
            'Highway MPG': HighwayMPG,
            'Seller Type': SellerType,
            'Seller Street Address': SellerStreetAddress,
            'Seller City': SellerCity,
            'Seller State': SellerState,
            'Seller Zipcode': SellerZipcode,
            'Seller Phone Number': SellerPhoneNumber 
        }
        
        #insert this listed vehicle's data to MongoDB database 
        mycollection.insert_one(mydict)
        
    print('Row '+str(i)+' successfully inserted!\n')
        

Row 0 successfully inserted!

Row 1 successfully inserted!

Row 2 successfully inserted!

Row 3 successfully inserted!

Row 4 successfully inserted!

Row 5 successfully inserted!

Row 6 successfully inserted!

Row 7 successfully inserted!

Row 8 successfully inserted!

Row 9 successfully inserted!

Row 10 successfully inserted!

Row 11 successfully inserted!

Row 12 successfully inserted!

Row 13 successfully inserted!

Row 14 successfully inserted!

Row 15 successfully inserted!

Row 16 successfully inserted!

Row 17 successfully inserted!

Row 18 successfully inserted!

Row 19 successfully inserted!

Row 20 successfully inserted!

Row 21 successfully inserted!

Row 22 successfully inserted!

Row 23 successfully inserted!

Row 24 successfully inserted!

Row 25 successfully inserted!

Row 26 successfully inserted!

Row 27 successfully inserted!

Row 28 successfully inserted!

Row 29 successfully inserted!

Row 30 successfully inserted!

Row 31 successfully inserted!

Row 32 successfull

### Run Summary Stats to find meaningful insights

Find this vehicle's market average price by condition(New/Used/Certified Used)

In [44]:
file_name #print database name composed by user input

'bmw_530i_2017_2021_90001_50miles'

In [45]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Condition",
            "avgPrice": {"$avg": "$List Price"}
        }
    }
])

for r in result:
    print(r)

{'_id': 'Used', 'avgPrice': 37123.13970588235}
{'_id': 'New', 'avgPrice': 59595.0}
{'_id': 'Certified', 'avgPrice': 31860.78947368421}


Find this vehicle's average listing price by Seller Type

In [46]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Seller Type",
            "Average Price": {"$avg": "$List Price"}
        }
    }
])

for r in result:
    print(r)

{'_id': 'Dealer', 'Average Price': 35065.15948275862}


Find how many dealers and how many private sellers who sell this vehicle

In [47]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Seller Type",
            "Count": {"$sum": 1}
        }
    }
])

for r in result:
    print(r)

{'_id': 'Dealer', 'Count': 232}


Find this vehicle's market average price by exterior color (Limit to Top 10 by price)

In [48]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Exterior Color",
            "Average Price": {"$avg": "$List Price"}
        }
    },
    { "$sort": { "Average Price": -1 } },
    { "$limit": 10 }
])

for r in result:
    print(r)

{'_id': '0416 Carbon Black Metallic', 'Average Price': 59615.0}
{'_id': '0c10 Mediterranean Blue Metallic', 'Average Price': 58305.0}
{'_id': 'Phytonic Blue Metallic', 'Average Price': 57594.0}
{'_id': 'Bernina Grey Amber Effect', 'Average Price': 53793.0}
{'_id': 'Mineral White', 'Average Price': 53282.5}
{'_id': 'Bluestone', 'Average Price': 45835.0}
{'_id': 'Glacier Silver', 'Average Price': 43182.5}
{'_id': 'Mediterranean Blue Metallic', 'Average Price': 42784.666666666664}
{'_id': 'Glacier Silver Metallic', 'Average Price': 38336.71428571428}
{'_id': 'Mineral White Metallic', 'Average Price': 37487.57142857143}


Find what exterior colors are popular in the market

In [49]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "$Exterior Color",
            "Count": {"$sum": 1}
        }
    },
    { "$sort": { "Count": -1 } }
])

for r in result:
    print(r)

{'_id': 'Alpine White', 'Count': 38}
{'_id': 'Jet Black', 'Count': 34}
{'_id': 'Bluestone Metallic', 'Count': 27}
{'_id': 'Glacier Silver Metallic', 'Count': 14}
{'_id': 'Dark Graphite Metallic', 'Count': 13}
{'_id': 'Black Sapphire Metallic', 'Count': 12}
{'_id': 'Silver', 'Count': 9}
{'_id': 'Black', 'Count': 9}
{'_id': 'Gray', 'Count': 8}
{'_id': 'White', 'Count': 8}
{'_id': 'Mineral White Metallic', 'Count': 7}
{'_id': 'Mediterranean Blue Metallic', 'Count': 6}
{'_id': 'Dark Graphite', 'Count': 6}
{'_id': 'Blue', 'Count': 4}
{'_id': 'Cashmere Silver Metallic', 'Count': 4}
{'_id': 'Atlas Cedar Metallic', 'Count': 4}
{'_id': 'Imperial Blue Metallic', 'Count': 4}
{'_id': 'Carbon Black Metallic', 'Count': 3}
{'_id': 'None', 'Count': 3}
{'_id': 'Glacier Silver', 'Count': 2}
{'_id': 'Bluestone', 'Count': 2}
{'_id': 'Phytonic Blue Metallic', 'Count': 2}
{'_id': 'Bernina Grey Amber Effect', 'Count': 2}
{'_id': 'Mineral White', 'Count': 2}
{'_id': 'White,Mineral', 'Count': 1}
{'_id': '0416 

Find this vehicle's average listing price from dealers that have ratings higher than 4.0

In [50]:
result = mycollection.aggregate([
    { "$match": { "Seller Rating": { "$gte": 4.0} } },
    {
        "$group": {
            "_id": "4.0+",
            "Average Price": {"$avg": "$List Price"}
        }
    },
    { "$sort": { "Average Price": -1 } }
])

for r in result:
    print(r)


{'_id': '4.0+', 'Average Price': 35780.550264550264}


Find dealer's average rating

In [51]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": "Dealer Rating",
            "Average Rating": {"$avg": "$Seller Rating"}
        }
    },
    { "$sort": { "Average Rating": -1 } }
])

for r in result:
    print(r)

{'_id': 'Dealer Rating', 'Average Rating': 4.588947368421053}


Find the seller contact information with price ascending

In [52]:
result = mycollection.aggregate([
    {
        "$group": {
            "_id": {
                "Seller Name":"$Seller Name",
                "Seller Phone Number":"$Seller Phone Number"
            },
            "Min Price": {"$min": "$List Price"}
        }
    },
    { "$sort": { "Min Price": 1 } }
])

for r in result:
    print(r)
    break

{'_id': {'Seller Name': 'Long Beach BMW', 'Seller Phone Number': '(562) 270-6592'}, 'Min Price': 24496}
